# Computing sequence score for scanpaths( Needleman-Wunsch algorithm): 

To compute the Sequence Score, the Needleman-Wunsch algorithm was used to calculate the minimum number of operations needed to change one string into another. 🚀 

📝  Notes:
- The sequence score is computed for scanpath of the same Visualization across different users
- We will compute sequence scores across the users of the same group as the view the same image

Needleman-Wunsch algorithm

In [5]:
def zero_one_similarity(a, b):
    if a == b:
        return 1.0
    else:
        return 0.0

def nw_matching(pred_string, gt_string, gap=0.0):
    # NW string matching with zero_one_similarity
    F = np.zeros((len(pred_string) + 1, len(gt_string) + 1), dtype=np.float32)
    for i in range(1 + len(pred_string)):
        F[i, 0] = gap * i
    for j in range(1 + len(gt_string)):
        F[0, j] = gap * j
    for i in range(1, 1 + len(pred_string)):
        for j in range(1, 1 + len(gt_string)):
            a = pred_string[i - 1]
            b = gt_string[j - 1]
            match = F[i - 1, j - 1] + zero_one_similarity(a, b)
            delete = F[i - 1, j] + gap
            insert = F[i, j - 1] + gap
            F[i, j] = np.max([match, delete, insert])
    score = F[len(pred_string), len(gt_string)]
    return score / max(len(pred_string), len(gt_string))

Finding the sequence scores

Creating plot showing sequence score of users in the same group

In [2]:
import glob

def list_images( group :int, mode:str, strings_path:str):
    """Generates the list of images for a given group

    Args:
        group (int): Can have Values from 1 to 10
        mode (str): Can be 'recall' or 'recognition'
        strings_path (str): path to strings folder --> VisQA\eyetracking\csv_files\strings

    Returns:
        list : list of images per group
    """
    for files in glob.glob(strings_path + '/*' + str(group) ):
        if os.path.exists(files + "/"+ mode):
            list_img_targets = os.listdir(files + "/"+ mode)
    return list_img_targets

In [96]:
def get_user_string_image(imname:str,username:str,strings_path:str, mode:str): 
    """Parses the string for the given user and image
    Args:
        imname (str): name of the image string file ending with .txt
        username (str): name of the user file 
        strings_path (str): path to strings folder --> VisQA\eyetracking\csv_files\strings
        mode (str): Can be 'recall' or 'recognition'

    Returns:
        list: scanpth of the user for the given image as a string
    """
    path = strings_path + '/' + username + '/'+ mode
    gtstring = os.path.join(path, imname)
    # print(gtstring)
    if os.path.exists(gtstring):
        f = open(gtstring,'r')
        scanpath_string = f.readline()
    else: 
        scanpath_string = None
        
    return scanpath_string

Change the group number and mode to plot the matrix ( mode can be only "recall", "recognition" )

In [115]:
group= 10
mode='recognition'

In [116]:
import numpy as np
import os, sys
sys.path.append("..") 
import pandas as pd
from tqdm import tqdm
# parameters
df_group_sequence_score = pd.DataFrame()

strings_path = r"C:\Users\karth\Desktop\Karthik\HCI_Lab\VisQA_new\eyetracking\csv_files\strings"



users_in_strings = os.listdir(strings_path)


users_in_group = [user for user in users_in_strings if user.endswith(str(group))]
list_img_targets = list_images(group= group, mode=mode, strings_path = strings_path)
## opening the exclude names
with open(r'C:\Users\karth\Desktop\Karthik\HCI_Lab\group4\dataset\excluded.csv') as file:
    excluded_names = file.read().splitlines()

users_in_group = [user for user in users_in_group if (user.endswith(str(group)) and (excluded_names.count(user) == 0))]
print(users_in_group)
## Create Seqeunce score matrix
seqscore_mat = np.zeros(shape=(len(users_in_group),len(users_in_group)))
i = 0
for user1 in tqdm(users_in_group):  
        j = 0
        for user2 in users_in_group:
                seqeunce_score = [] 
                for imgname in list_img_targets: 
                    user1_string = get_user_string_image(imgname, user1, strings_path, mode)
                    user2_string = get_user_string_image(imgname, user2, strings_path, mode)
                    res = nw_matching(user2, user1)
                    seqeunce_score.append(res)
                mean = np.mean(np.array(seqeunce_score))
                seqscore_mat[i][j] = mean
                j = j+1       
        i = i+1 


"""__Plotting part__copy"""
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_theme(style="white", font_scale=4)

cmap = sns.diverging_palette(230, 20, as_cmap=True)


cc_matrix = seqscore_mat
# creating a dictonary to store the Values 
pd_dict = {}
person_num_list = []
for i in range(len(seqscore_mat)):
    person_num = 'p' + str(i)
    person_num_list.append(person_num)
    pd_dict[person_num] = seqscore_mat[i]

ccpd = pd.DataFrame(pd_dict)

ccpd.index = person_num_list
mask = np.triu(np.ones_like(ccpd, dtype=bool))

f, ax = plt.subplots(figsize=(16, 16), dpi=600)
sns.heatmap(ccpd, annot=False, mask=mask, linewidths=.5, ax=ax, cmap=cmap, vmin=0, vmax=.5, cbar=True)
plt.title(f"Group{group}")
if mode == 'recall':
    stage = 'encoding'
else:
    stage = 'recognition'    
plt.savefig(f"Seqscore_Mat_Group{group}{stage}")               

['Boya10', 'chuang10', 'guanzh10', 'haotia10', 'Junmao10', 'Marie10', 'purong10', 'Simhad10', 'sunxin10', 'xingyu10', 'yuru10', 'zihao10']


100%|██████████| 12/12 [00:05<00:00,  2.17it/s]


In [117]:
# import seaborn as sns
# import matplotlib.pyplot as plt

# sns.set_theme(style="white", font_scale=4)

# cmap = sns.diverging_palette(230, 20, as_cmap=True)


# cc_matrix = seqscore_mat
# # creating a dictonary to store the Values 
# pd_dict = {}
# person_num_list = []
# for i in range(len(seqscore_mat)):
#     person_num = 'p' + str(i)
#     person_num_list.append(person_num)
#     pd_dict[person_num] = seqscore_mat[i]

# ccpd = pd.DataFrame(pd_dict)

# ccpd.index = person_num_list
# mask = np.triu(np.ones_like(ccpd, dtype=bool))

# f, ax = plt.subplots(figsize=(16, 16), dpi=600)
# sns.heatmap(ccpd, annot=False, mask=mask, linewidths=.5, ax=ax, cmap=cmap, vmin=0, vmax=.5, cbar=True)
# plt.title(f"Group{group}")
# if mode == 'recall':
#     stage = 'encoding'
# else:
#     stage = 'recognition'    
# plt.savefig(f"Seqscore_Mat_Group{group}{stage}")